# Biopython
-------------------------------------------------

BioPython is a very popular third party Python package for handling biological data.

In order to install it, type in a **terminal** window:

`bash`

`conda install -c anaconda biopython`

and follow the instructions at prompt

You can check the basic installation and inspect the version by doing:

In [ ]:
import Bio
print(Bio.__version__)

Biopython has three major functionalities
-  Sequence handling
-  3D structure
-  Population Genetics

Some useful references: __[Website](https://biopython.org/)__, __[Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html)__    (most of the examples are coming from these sources)

# Working with Sequences in BioPython

 -  Reading sequence files
 -  Writing sequence files
 -  Working with sequence features
 -  Sequence alignments

Sequences are just strings of letters. In principle, we could deal with them defining normal python strings.
However, BioPython has some nice features that helps us distinguishing those strings as belonging to DNA, RNA or proteins and working with them.

## BioPython.Seq

Sequences are handled by the __[Seq](https://biopython.org/wiki/Seq)__ class. 

In [ ]:
#let's make a generic sequence
from Bio.Seq import Seq


In [ ]:
help(Seq)

In [ ]:
my_seq = Seq("CCCAGGGAG")

In [ ]:
print(type(my_seq)) #print the type of the object

#let's see what attributes this object has
dir(my_seq) #prints all the attribute of an object


attributes = [a for a in dir(my_seq) if not a.startswith("_")] #
print(attributes)


The alphabet object is perhaps the important thing that makes the Seq object more than just a string.

In [ ]:
my_seq.alphabet

The currently available alphabets for Biopython are defined in the Bio.Alphabet module

In [ ]:
from Bio.Alphabet import generic_dna, generic_protein, generic_rna

In [ ]:
my_dna = Seq("CCCGGAGAG", generic_dna)
my_rna = Seq("ACCCGUUGU", generic_rna)
my_protein = Seq("AKMARKHLNL", generic_protein)

Biopython will now know the difference for example between a <font color=blue>DNA</font> base <font color=blue>A</font> for adenine and the <font color=red>protein</font> residue <font color=red>A</font> for alanine.

In [ ]:
print(my_dna.alphabet)

Let's look at how to utilize this class to do interesting work as, for example, transcribe and translate a sequence 

In [ ]:
my_gene = Seq("ACTAGCAGCGGA", generic_dna)

#get the mRNA

my_transcript = my_gene.transcribe()
print(my_transcript)
print(my_transcript.alphabet)


#get the protein from the mRNA
my_protein = my_transcript.translate()
print(my_protein)
print(my_protein.alphabet)

## Sequences act like strings

In many ways, we can deal with Seq objects as if they were normal Python strings, for example getting the length, or iterating over the elements:

In [ ]:
my_seq = Seq("GATCG", generic_dna)

You can access elements of the sequence in the same way as for strings (but remember, Python counts from zero!):

In [ ]:
print(my_seq[0]) #first letter
print(my_seq[2]) #third letter
print(my_seq[-1]) #last letter

Slicing a sequence:

In [ ]:
my_seq = Seq("GATCGATGGGCCTATATAGGATCGAAAATCGC", generic_dna)
my_seq[5:11]

Two things are interesting to note:
- First, this follows the normal conventions for Python strings. So the first element of the sequence is 0. When you do a slice the first item is included (i.e. 5 in this case) and the last is excluded (11 in this case)
- The new object produced is another Seq object which retains the alphabet information from the original Seq object.

The Seq object has a .count() method, just like a string. Note that this means that like a Python string, this gives a non-overlapping count:

In [ ]:
print("AAAA".count("AA"))
print(Seq("AAAA").count("AA"))

Naturally, you can in principle add any two Seq objects together - just like you can with Python strings to concatenate them. However, you **can't add sequences with incompatible alphabets**, such as a protein sequence and a DNA sequence:

In [ ]:
protein_seq = Seq("EVRNAK", generic_protein)
dna_seq = Seq("ACGT", generic_dna)

protein_seq + dna_seq

In [ ]:
protein_seq = Seq("EVRNAK", generic_protein)
protein2_seq = Seq("RAV", generic_protein)


protein_seq + protein2_seq 


You can also go straight from the DNA to the protein.

<a id="section1"></a>

In [ ]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG", generic_dna)
myprot = coding_dna.translate()
print(myprot)

As you can see, we got some STOP codons represented as * and translation continued.

We can get translation to actually stop when it encounters a STOP codon.

In [ ]:
myprot = coding_dna.translate(to_stop=True)
print(myprot)

We can be even more realistic and only allow translation of valid genes (i.e. with a valid start and stop codon and proper number of bases).

This is done by setting the ** cds=True ** keyword argument, which stands for "coding sequence".

If we don't have a valid coding sequence, we get an exception.

In [ ]:
myprot = coding_dna.translate(cds=True)

Partial codons are always regarded as an error!

In [ ]:
gene = Seq("ATGGCCATTGTAATGTAG", generic_dna)
gene.translate(cds=True)



# Reading sequence files 

Dealing with assorted sequence file formats is one of the strengths of Biopython.The primary module we'll be using is __[Seq.IO](https://biopython.org/wiki/SeqIO)__, which is short for sequence input/output.

For these examples we're going to use files for the famous bacteria Esherichia coli K12, and some potato genes.
To download the sample data needed for this section use the  fetch_sample_data.sh script.

In [ ]:
! ./fetch_sample_data.sh


We'll start by looking at the protein sequence in the FASTA amino acid file, NC_000913.faa. 

The annotation for a FASTA  sequence is tipically held in the header:

In [ ]:
! head NC_000913.faa

The SeqIO.parse takes a path to a file and a format, in this case "fasta" and produces an iterator over each entry in the file

In [ ]:
from Bio import SeqIO

In [ ]:
filename = "NC_000913.faa"
for record in SeqIO.parse(filename, "fasta"):
    print(type(record))
    

<div class="alert alert-success">

**Exercise 1:**  count the records with Biopython using the SeqIO.parse function and print how many records you have in NC_000913.faa

</div>

In [ ]:
# Solution:


## Looking at the Records

In the above example, we used a for loop to count the records in a FASTA file, but didn't actually look at the information in the records. 
The **SeqIO.parse** function was creating **SeqRecord** objects. 
Biopython's SeqRecord objects are a container holding the sequence, and any annotation about it - most importantly the identifier.

In [ ]:
print(record.id)

In [ ]:
print (record.seq)

 This simple example prints out the record identifers and their lengths:

In [ ]:
from Bio import SeqIO
filename = "NC_000913.faa"
for record in SeqIO.parse(filename, "fasta"):
    print("Record " + record.id + ", length " + str(len(record.seq)))

<div class="alert alert-success">
**Excercise 2:** - Warm up - Count how many sequences are less than 100 amino acids long

<div>

In [ ]:
#Solution



 <div class="alert alert-success">
 **Exercise 3:** - Warm up - Plot a histogram of the sequence length distribution
 <div>

In [ ]:
#Solution



In the next example we'll check all the protein sequences start with a methionine (represented as the letter "M" in the standard IUPAC single letter amino acid code), and count how many records fail this.



In [ ]:
from Bio import SeqIO
filename = "NC_000913.faa"
bad = 0
for record in SeqIO.parse(filename, "fasta"):
    if not record.seq.startswith("M"):
        bad = bad + 1
        print(record.id + " starts " + record.seq[0])
print("Found " + str(bad) + " records in " + filename + " which did not start with M")

 <div class="alert alert-success">
**Excercise 4**: Modify the script above  to print out the description of the problem records, not just the identifier for the the potato protein file *PGSC_DM_v3.4_pep_representative.fasta*.


- What did you notice about these record descriptions? 
- Can you think of any reasons why there could be so many genes/proteins with a problem at the start?

<div>

In [ ]:
#Solution

## Different File Formats

The Biopython SeqIO module supports quite a few other important sequence file formats (see the table on the SeqIO wiki page).

If you work with genomes, you'll probably have to deal with annotated files in the EMBL or GenBank format. Let's try this with the E. coli K12 GenBank file, NC_000913.gbk, based on the previous example:

In [ ]:
import timeit
from Bio import SeqIO
start_time = timeit.default_timer()
fasta_record = SeqIO.read("NC_000913.fna", "fasta")
print(fasta_record.id + " length " + str(len(fasta_record)))
elapsed = timeit.default_timer() - start_time
print("time:",elapsed)

In [ ]:
import timeit
from Bio import SeqIO
start_time = timeit.default_timer()
genbank_record = SeqIO.read("NC_000913.gbk", "genbank")
print(genbank_record.id + " length " + str(len(genbank_record)))
elapsed = timeit.default_timer() - start_time
print("time:", elapsed)

All we needed to change was the file format argument to the SeqIO.read(...) function - and we could load a GenBank file instead. 
You'll notice the GenBank version was given a shorter identifier, and took longer to load. 
The reason is that there is a lot more information present - most importantly lots of features (where each gene is and so on). We'll return to this in a later section (working with sequence features).

# Writing sequence files 

We can also write our own sequence using the  **Bio.SeqIO.write()** function.

This is a function taking three arguments:
some SeqRecord objects, a handle or filename to write to, and a sequence format.

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_protein

record1 = SeqRecord(Seq("ALKHTYIKKKLLLMN", generic_protein), id ="1|1|", description ="protein 1")
record2 = SeqRecord(Seq("LTHKLLL", generic_protein),id="2|2|" ,description = "protein 2")

my_records = [record1, record2]

from Bio import SeqIO
SeqIO.write(my_records, "my_example.faa", "fasta")

In [ ]:
! head -10 my_example.faa


# Accessing public databases

If, rather than getting sequence record from a file, we wish to get data directly from a database then there are a few helper functions in BioPython that allow easy access to some large database

1. For example, we can read a SWISSPROT record using ExPASy:

In [ ]:
from Bio import ExPASy

In [ ]:
from Bio import SeqIO
socketObj = ExPASy.get_sprot_raw('ACM1_HUMAN')
proteinObj = SeqIO.read(socketObj, "swiss")
socketObj.close()

In [ ]:
print(proteinObj.description)
print(proteinObj.seq)
print(proteinObj.name)
print("Length %i" % len(proteinObj))

2. Entrez (http://www.ncbi.nlm.nih.gov/Entrez) is a data retrieval system that provides users access to NCBI’s databases such as PubMed, GenBank, GEO, and many others. 

You can access Entrez from a web browser to manually enter queries, or you can use Biopython’s Bio.Entrez module for programmatic access to Entrez. 
The latter allows you for example to search PubMed or download GenBank records from within a Python script.


The Bio.Entrez module makes use of the Entrez Programming Utilities (also known as EUtils), consisting of eight tools that are described in detail on NCBI’s page at http://www.ncbi.nlm.nih.gov/entrez/utils/. 

Each of these tools corresponds to one Python function in the Bio.Entrez module (for more information: https://biopython.readthedocs.io/en/latest/Tutorial/chapter_entrez.html). 

This module makes sure that the correct URL is used for the queries, and that not more than one request is made every three seconds, as required by NCBI.

The output returned by the Entrez Programming Utilities is typically in XML format.

In [ ]:
from Bio import Entrez
#Please, change the e-mail address below!!!
Entrez.email = "name.surname@fau.de" #always tell NCBI who you are



**EInfo** provides field index term counts, last update, and available links for each of NCBI’s databases. In addition, you can use EInfo to obtain a list of all database names accessible through the Entrez utilities:

In [ ]:
handle = Entrez.einfo()
result = handle.read()
handle.close()

The variable result now contains a list of databases in XML format:

In [ ]:
print(result)

In [ ]:
Using Bio.Entrez’s parser, we can directly parse this XML file into a Python object:

In [ ]:
handle = Entrez.einfo()
record = Entrez.read(handle)

Now record contains **keys** which are the list of database names shown in the XML :

In [ ]:
record["DbList"]

To search any of these databases, we use Bio.Entrez.esearch(). For example, let’s search in PubMed for publications related to Biopython:

In [ ]:
handle = Entrez.esearch(db="pubmed", term="biopython")
record = Entrez.read(handle)

In [ ]:
print(record["IdList"])

In this output, you see lots of PubMed IDs which can be retrieved by EFetch. 

**EFetch** is what you use when you want to retrieve a full record from Entrez.


For most of their databases, the NCBI support several different file formats. Requesting a specific file format from Entrez using Bio.Entrez.efetch() requires specifying the **rettype** and/or **retmode** optional arguments.
The different combinations are described for each database type on the pages linked to on NCBI efetch webpage (e.g. literature, sequences and taxonomy).

One common usage is downloading sequences in the FASTA or GenBank/GenPept plain text formats (which can then be parsed with Bio.SeqIO. From the Cypripedioideae, we can download GenBank record EU490707 using Bio.Entrez.efetch:

In [ ]:
handle = Entrez.efetch(db="nucleotide", id="EU490707", rettype="gb", retmode="text")
print(handle.read())

The arguments rettype="gb" and retmode="text" let us download this record in the GenBank format. 
Alternatively, you could for example use rettype="fasta" to get the Fasta-format. Remember – the available formats depend on which database you are downloading from - see the main EFetch Help (https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch) page. 

If you fetch the record in one of the formats accepted by Bio.SeqIO, you could directly parse it into a SeqRecord

In [ ]:
filename = "EU490707.gbk"
net_handle = Entrez.efetch(db="nucleotide", id="EU490707", rettype="gb", retmode="text")
out_handle = open(filename, "w")
out_handle.write(net_handle.read())
out_handle.close()
net_handle.close()
print("Saved")

record = SeqIO.read(filename, "genbank")
print(record)

# Working with sequence features

Most of the time GenBank files contain a single record for a single chromosome or plasmid, so we'll generally use the SeqIO.read(...) function. Remember the second argument is the file format, so if we start from the code to read in a FASTA file:

In [ ]:
from Bio import SeqIO

record = SeqIO.read("NC_000913.fna", "fasta")
print(record.id)
print(len(record))
print(len(record.features))


Now switch the filename and the format:

In [ ]:
record = SeqIO.read("NC_000913.gbk", "genbank")
print(record.id)
print(len(record))
print(len(record.features))


So what is this new **.features** thing? 
It is a Python list, containing a Biopython SeqFeature object for each feature in the GenBank file. For instance,

In [ ]:
my_gene = record.features[3]
print(my_gene)

There are three key properties, **.type** which is a string like gene or CDS, **.location** which describes where on the genome this feature is, and **.qualifiers** which is a Python dictionary full of all the annotation for the feature (things like gene identifiers).

## Feature Locations

In [ ]:
from Bio import SeqIO
record = SeqIO.read("NC_000913.gbk", "genbank")
my_gene = record.features[3]
print(my_gene.qualifiers["locus_tag"])

print(my_gene.location)

print(my_gene.location.start)

print(my_gene.location.end)

print(my_gene.location.strand)


This was a very simple location on the forward strand, if it had been on the reverse strand you'd need to take the reverse-complement. Also if the location had been a more complicated compound location like a join (used for eukaryotic genes where the CDS is made up of several exons), then the location would have-sub parts to consider.

All these complications are taken care of for you via the .extract(...) method which takes the full length parent record's sequence as an argument:

In [ ]:
gene_seq = my_gene.extract(record.seq)
len(gene_seq)
print(gene_seq)

<div class="alert alert-success">
**Exercise 5:** Finish the following script by setting an appropriate feature name like the locus tag or GI number (use the .qualifiers or .dbxrefs information) to extract all the coding sequences from the GenBank file:
<div>

In [ ]:
from Bio import SeqIO
record = SeqIO.read("NC_000913.gbk", "genbank")
output_handle = open("NC_000913_cds.fasta", "w")
count = 0
for feature in record.features:
    if feature.type == "CDS":
        count = count + 1
        feature_name = "..." # Use feature.qualifiers or feature.dbxrefs here
        feature_seq = feature.extract(record.seq)
        # Simple FASTA output without line wrapping:
        output_handle.write(">" + feature_name + "\n" + str(feature_seq) + "\n")
output_handle.close()
print(str(count) + " CDS sequences extracted")

## Feature Lengths


The length of Biopython's SeqFeature objects (and the location objects) is defined as the length of the sequence region they describe (i.e. how many bases are included; or for protein annotation how many amino acids).

In [ ]:
from Bio import SeqIO
record = SeqIO.read("NC_000913.gbk", "genbank")
print("Total length of genome is " + str(len(record)))

<div class="alert alert-success">
** Exercise 6**: Write a code to calculate to calculate the total length of the genes and of the coding sequences (CDS) within the NC_000913.gbk genome. Give a separate count for the "gene" and "CDS" feature type. 
<div>

In [ ]:
#Solution



What proportion of the genome is annotated as gene coding?

In [ ]:
#Solution:


<div class="alert alert-success">
**Exercise 7:** Extend the previous script to also count the number of features of each type, and report this and the average length of that feature type. e.g.
<div>

In [ ]:
#Solution


# Sequence alignments

The **AlignIO** module which as the name suggests is for alignment input and ouput. 
This is focused on dealing with multiple sequence alignments of the kind typically used in phylogenetics - a separate **SearchIO** module targets pairwise alignments generated by search tools like BLAST.

We're going to look at a small seed alignment for one of the PFAM domains, the A2L zinc ribbon domain (A2L_zn_ribbon; PF08792). This was picked almost at random - it is small enough to see the entire alignment on screen, and has some obvious gap-rich columns.

From the alignments tab on the Pfam webpage, you can download the raw alignment in several different formats (Selex, Stockholm, FASTA, and MSF). Biopython is able to work with FASTA (very simple) and Stockholm format (richly annotated).

In [ ]:
from Bio import AlignIO

alignment = AlignIO.read("PF08792_seed.sth", "stockholm")

As you might guess from using SeqIO.convert(...) and SeqIO.write(...), there are matching AlignIO.convert() and AlignIO.write(...) functions.

In [ ]:
from Bio import AlignIO
input_filename = "PF08792_seed.sth"
output_filename = "PF08792_seed_converted.fasta"
AlignIO.convert(input_filename, "stockholm", output_filename, "fasta")

In [ ]:
! head -10 PF08792_seed_converted.fasta


<div class="alert alert-success">
** Excercise 8:** Pairwise Sequence identity:

- Calculate the sequence identity (expressed in percentage) between every pair of sequences of the PF08792_seed_converted.fasta data set. 

- Calculate the mean and the standard deviation of the percentages

- Filter the pairs of sequences that have a sequence identity (statistically) significantly deviating from the mean

- Save on a fasta file the filtered sequence ordered according to their sequence identity percentage
<div>

For this exercise you might find useful to know that:

In [ ]:
a=np.array([0,5,7,4,0])
print(a)
print(a[:]==0)
print(a[a[:]==0])

In [ ]:
#solution 





Sequence pairs in an alignment can have a score, even when they are not identical,  which describe how similar two sequences are. This similarity is defined considering how *substitutable* one residue type is for another; in other words how likely they are to have been swapped or exchanged for one another.
Residues that commonly swap for one another are deemed to be similar and give high scores, while those that rarely swap are dissimilar and give low scores.


The substitutability of one residue for another is stored in a two-dimensional array, commonly called **substitution matrix** or **similarity matrix**.

We will define our similarity matrix using a **python dictionary**

A **dictionary** is a collection which is unordered, changeable and indexed. 
In Python dictionaries are written with curly brackets, and they have keys and values.
For example:

In [ ]:
thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}
print(thisdict)

You can access the items of a dictionary by referring to its key name:

In [ ]:
x = thisdict["model"]
x

Or also use the values() function to return values of a dictionary:

In [ ]:
for x in thisdict.values():
    print(x)

Adding an item to the dictionary is done by using a new index key and assigning a value to it:

In [ ]:
thisdict ={
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}
thisdict["color"] = "red"
print(thisdict)

There are several methods to remove items from a dictionary:

1. The **del** and the **pop()** keyword both remove the item with the specified key name:

In [ ]:
thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}
del thisdict["model"]
print(thisdict)

In [ ]:
thisdict = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}
thisdict.pop("model")
print(thisdict)

The del keyword can also delete the dictionary completely:

In [ ]:
thisdict ={
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}
del thisdict
print(thisdict) #this will cause an error because "thislist" no longer exists.

2. The **popitem()** method removes the last inserted item 

In [ ]:
thisdict ={
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}
thisdict.popitem()
print(thisdict)

3. The **clear ()** method empties the dictionary:

In [ ]:
thisdict ={
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}
thisdict.clear()
print(thisdict)

Below there are examples of substitution matrices.

* The first one - DNA_1 -is very simple and would give scores as if you were measuring sequence identity (i.e., a score of 1 were residues are identical, 0 elsewhere.). 
* Rather than scoring DNA for matches we could also score for complementarity (i.e., using Watson-Crick's pairing rules), with 1 for A:T or C:G matches 
and -1 for mismatches. Expressed as Python dictionary this would correspond to the example called REV_COMP
*  Going to a more complex example, in the DNA_2 matrix identicatl residues give a score of 1 but non identitcal -3. The latter indicate that there is a mismatch between the sequence. A score of 0 is here used to indicate indifference
* The last example, raprestents one of the most widely used similarity matrix, called BLOSUM62 (to read more about it: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC50453/pdf/pnas01096-0363.pdf ).                                                                               

In [ ]:
DNA_1 = {'G': { 'G':1, 'C':0, 'A':0, 'T':0 },
         'C': { 'G':0, 'C':1, 'A':0, 'T':0 },
         'A': { 'G':0, 'C':0, 'A':1, 'T':0 },
         'T': { 'G':0, 'C':0, 'A':0, 'T':1 }}

          
REV_COMP = {'G': { 'G':-1, 'C': 1, 'A':-1, 'T':-1 },
            'C': { 'G': 1, 'C':-1, 'A':-1, 'T':-1 },
            'A': { 'G':-1, 'C':-1, 'A':-1, 'T': 1 },
            'T': { 'G':-1, 'C':-1, 'A': 1, 'T':-1 }}

          
DNA_2 = {'G': { 'G': 1, 'C':-3, 'A':-3, 'T':-3, 'N':0 },
         'C': { 'G':-3, 'C': 1, 'A':-3, 'T':-3, 'N':0 },
         'A': { 'G':-3, 'C':-3, 'A': 1, 'T':-3, 'N':0 },
         'T': { 'G':-3, 'C':-3, 'A':-3, 'T': 1, 'N':0 },
         'N': { 'G': 0, 'C': 0, 'A': 0, 'T': 0, 'N':0 }}  

BLOSUM62 = {'A':{'A': 4,'R':-1,'N':-2,'D':-2,'C': 0,'Q':-1,'E':-1,'G': 0,'H':-2,'I':-1,
                 'L':-1,'K':-1,'M':-1,'F':-2,'P':-1,'S': 1,'T': 0,'W':-3,'Y':-2,'V': 0,'X':0},
            'R':{'A':-1,'R': 5,'N': 0,'D':-2,'C':-3,'Q': 1,'E': 0,'G':-2,'H': 0,'I':-3,
                 'L':-2,'K': 2,'M':-1,'F':-3,'P':-2,'S':-1,'T':-1,'W':-3,'Y':-2,'V':-3,'X':0},
            'N':{'A':-2,'R': 0,'N': 6,'D': 1,'C':-3,'Q': 0,'E': 0,'G': 0,'H': 1,'I':-3,
                 'L':-3,'K': 0,'M':-2,'F':-3,'P':-2,'S': 1,'T': 0,'W':-4,'Y':-2,'V':-3,'X':0},
            'D':{'A':-2,'R':-2,'N': 1,'D': 6,'C':-3,'Q': 0,'E': 2,'G':-1,'H':-1,'I':-3,
                 'L':-4,'K':-1,'M':-3,'F':-3,'P':-1,'S': 0,'T':-1,'W':-4,'Y':-3,'V':-3,'X':0},
            'C':{'A': 0,'R':-3,'N':-3,'D':-3,'C': 9,'Q':-3,'E':-4,'G':-3,'H':-3,'I':-1,
                 'L':-1,'K':-3,'M':-1,'F':-2,'P':-3,'S':-1,'T':-1,'W':-2,'Y':-2,'V':-1,'X':0},
            'Q':{'A':-1,'R': 1,'N': 0,'D': 0,'C':-3,'Q': 5,'E': 2,'G':-2,'H': 0,'I':-3,
                 'L':-2,'K': 1,'M': 0,'F':-3,'P':-1,'S': 0,'T':-1,'W':-2,'Y':-1,'V':-2,'X':0},
            'E':{'A':-1,'R': 0,'N': 0,'D': 2,'C':-4,'Q': 2,'E': 5,'G':-2,'H': 0,'I':-3,
                 'L':-3,'K': 1,'M':-2,'F':-3,'P':-1,'S': 0,'T':-1,'W':-3,'Y':-2,'V':-2,'X':0},
            'G':{'A': 0,'R':-2,'N': 0,'D':-1,'C':-3,'Q':-2,'E':-2,'G': 6,'H':-2,'I':-4,
                 'L':-4,'K':-2,'M':-3,'F':-3,'P':-2,'S': 0,'T':-2,'W':-2,'Y':-3,'V':-3,'X':0},
            'H':{'A':-2,'R': 0,'N': 1,'D':-1,'C':-3,'Q': 0,'E': 0,'G':-2,'H': 8,'I':-3,
                 'L':-3,'K':-1,'M':-2,'F':-1,'P':-2,'S':-1,'T':-2,'W':-2,'Y': 2,'V':-3,'X':0},
            'I':{'A':-1,'R':-3,'N':-3,'D':-3,'C':-1,'Q':-3,'E':-3,'G':-4,'H':-3,'I': 4,
                 'L': 2,'K':-3,'M': 1,'F': 0,'P':-3,'S':-2,'T':-1,'W':-3,'Y':-1,'V': 3,'X':0},
            'L':{'A':-1,'R':-2,'N':-3,'D':-4,'C':-1,'Q':-2,'E':-3,'G':-4,'H':-3,'I': 2,
                 'L': 4,'K':-2,'M': 2,'F': 0,'P':-3,'S':-2,'T':-1,'W':-2,'Y':-1,'V': 1,'X':0},
            'K':{'A':-1,'R': 2,'N': 0,'D':-1,'C':-3,'Q': 1,'E': 1,'G':-2,'H':-1,'I':-3,
                 'L':-2,'K': 5,'M':-1,'F':-3,'P':-1,'S': 0,'T':-1,'W':-3,'Y':-2,'V':-2,'X':0},
            'M':{'A':-1,'R':-1,'N':-2,'D':-3,'C':-1,'Q': 0,'E':-2,'G':-3,'H':-2,'I': 1,
                 'L': 2,'K':-1,'M': 5,'F': 0,'P':-2,'S':-1,'T':-1,'W':-1,'Y':-1,'V': 1,'X':0},
            'F':{'A':-2,'R':-3,'N':-3,'D':-3,'C':-2,'Q':-3,'E':-3,'G':-3,'H':-1,'I': 0,
                 'L': 0,'K':-3,'M': 0,'F': 6,'P':-4,'S':-2,'T':-2,'W': 1,'Y': 3,'V':-1,'X':0},
            'P':{'A':-1,'R':-2,'N':-2,'D':-1,'C':-3,'Q':-1,'E':-1,'G':-2,'H':-2,'I':-3,
                 'L':-3,'K':-1,'M':-2,'F':-4,'P': 7,'S':-1,'T':-1,'W':-4,'Y':-3,'V':-2,'X':0},
            'S':{'A': 1,'R':-1,'N': 1,'D': 0,'C':-1,'Q': 0,'E': 0,'G': 0,'H':-1,'I':-2,
                 'L':-2,'K': 0,'M':-1,'F':-2,'P':-1,'S': 4,'T': 1,'W':-3,'Y':-2,'V':-2,'X':0},
            'T':{'A': 0,'R':-1,'N': 0,'D':-1,'C':-1,'Q':-1,'E':-1,'G':-2,'H':-2,'I':-1,
                 'L':-1,'K':-1,'M':-1,'F':-2,'P':-1,'S': 1,'T': 5,'W':-2,'Y':-2,'V': 0,'X':0},
            'W':{'A':-3,'R':-3,'N':-4,'D':-4,'C':-2,'Q':-2,'E':-3,'G':-2,'H':-2,'I':-3,
                 'L':-2,'K':-3,'M':-1,'F': 1,'P':-4,'S':-3,'T':-2,'W':11,'Y': 2,'V':-3,'X':0},
            'Y':{'A':-2,'R':-2,'N':-2,'D':-3,'C':-2,'Q':-1,'E':-2,'G':-3,'H': 2,'I':-1,
                 'L':-1,'K':-2,'M':-1,'F': 3,'P':-3,'S':-2,'T':-2,'W': 2,'Y': 7,'V':-1,'X':0},
            'V':{'A': 0,'R':-3,'N':-3,'D':-3,'C':-1,'Q':-2,'E':-2,'G':-3,'H':-3,'I': 3,
                 'L': 1,'K':-2,'M': 1,'F':-1,'P':-2,'S':-2,'T': 0,'W':-3,'Y':-1,'V': 4,'X':0},
            'X':{'A': 0,'R': 0,'N': 0,'D': 0,'C': 0,'Q': 0,'E': 0,'G': 0,'H': 0,'I': 0,
                 'L': 0,'K': 0,'M': 0,'F': 0,'P': 0,'S': 0,'T': 0,'W': 0,'Y': 0,'V': 0,'X':0}}

In [ ]:
The following function will now consider a substitution matrix and use it to calculate an overall similarity score. 

In [ ]:
def calcSeqSimilarity(seqA, seqB, simMatrix):

    numPlaces = min(len(seqA), len(seqB))
  
    totalScore = 0.0
  
    for i in range(numPlaces):
        residueA = seqA[i]
        residueB = seqB[i]
  
        totalScore += simMatrix[residueA][residueB]

    return totalScore

In [ ]:
# Test with pre-defined substitution matrices
  # DNA example
print(calcSeqSimilarity('AGCATCGCTCT', 'AGCATCGTTTT', DNA_2))

In [ ]:
#Protein example
print(calcSeqSimilarity ('ALIGNMENT', 'ALINGEMTN', BLOSUM62))

<div class="alert alert-success">
**Exercise 9:** Modify the above function to take into account the presence of gaps ('-') in the sequences without modifying the BLOSUM62 substitution matrix. 
Gaps are generally undesirable but are tolerable if the subsequent alignment matches well.

Introduce different gap penalties depending on whether there is an insertion of a new gap or an extension of an existing one. 

<div>

In [ ]:
#Solution



## Optimising pairwise alignment

We have seen the basic principles of how it is possible to measure the match quality of an aligned pair of sequences. 
In general, however, one is interested in determining which alignment out of all the possible combinations is the best (highest scoring).
Consider the following three examples:

The last alignment is the best, with the middle one a close second.

This problem is cleary more complex. The number of possible combinations grows very rapidly with the length of the sequence. Hence, more clever algorithm have been developed. All of them use the so called *dynamic programming* principle.


The idea behind dynamic programming is that a big problem can be broken down into local-sub-problems that occur repeatedly, then the solution to each sub-problem only needs to be calculated once. 

In terms of sequence alignment the big problem is to find the highest scoring arrangement and the local sub-problems involve smaller sub-alignments.

We will not cover this here, but we invite you to refer to Durbin et al. (https://pdfs.semanticscholar.org/2ed5/d6b35f8971fb9d7434a2683922c3bfcc058e.pdf) for in-depth information on sequence alignment algorithms.   

BioPython includes a built-in pairwise aligner that implements some of these algorithms: e.g. the Needleman-Wunsch, Smith-Waterman, Gotoh (three-state), and Waterman-Smith-Beyer global and local pairwise alignment algorithms. 

In [ ]:
from Bio import Align
from Bio.SubsMat.MatrixInfo import blosum62
aligner = Align.PairwiseAligner()

The PairwiseAligner object aligner  stores the alignment parameters to be used for the pairwise alignments.

In [ ]:
seqA = "ALIGDPPVENTS"
seqB = "ALIGN--MENTS"
aligner = Align.PairwiseAligner()
aligner.substitution_matrix = blosum62
aligner.extend_gap_score = -4
aligner.open_gap_score = -8
score = aligner.score(seqA, seqB)
print(score)

alignments = aligner.align(seqA, seqB)
len(alignments)
print(alignments[0].score)

print(alignments[0])

Depending on the gap scoring parameters and mode, a PairwiseAligner object automatically chooses the appropriate algorithm to use for pairwise sequence alignment. To verify the selected algorithm, use

In [ ]:
aligner.algorithm

In [ ]:
alignments = aligner.align(seqA, seqB)
for alignment in alignments:
    print(alignment)

## BLAST

BLAST (Basic Local Alignment Search Tool) is one of the most widely used bioinformatics programs for sequence searching within a library or database of sequences, and to identify library sequences that resemble the query sequence above a certain threshold (e-value).

Using a dynamic algorithm for this purpose would be very computational expensive. BLAST use a so called *heuristic* approach ( https://en.wikipedia.org/wiki/BLAST ) and it does **not** perform an optimal alignment. 

You can use BLAST through BioPython:
the function qblast() in the Bio.Blast.NCBIWWW module calls the online version of BLAST. 


* The first argument is the blast program to use for the search, as a lower case string. The options and descriptions of the programs are available at http://www.ncbi.nlm.nih.gov/BLAST/blast_program.shtml. Currently qblast only works with blastn, blastp, blastx, tblast and tblastx.

* The second argument specifies the databases to search against. Again, the options for this are available on the NCBI web pages at http://www.ncbi.nlm.nih.gov/BLAST/blast_databases.shtml.

* The third argument is a string containing your query sequence. This can either be the sequence itself, the sequence in fasta format, or an identifier like a GI number.



The qblast function also take a number of other option arguments which are basically analogous to the different parameters you can set on the BLAST web page.

* The argument url_base sets the base URL for running BLAST over the internet. By default it connects to the NCBI.

* The qblast function can return the BLAST results in various formats, which you can choose with the optional format_type keyword: "HTML", "Text", "ASN.1", or "XML". The default is "XML", as that is the format expected by the parser

* The argument expect sets the expectation or e-value threshold.


In [ ]:
from Bio.Blast import NCBIWWW
result_handle = NCBIWWW.qblast("blastn", "nt", "8332126")

In [ ]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)


In [ ]:
E_VALUE_THRESH = 0.04


for alignment in blast_record.alignments:
    for hsp in alignment.hsps:  # HSP (high-scoring pair) 
        if hsp.expect < E_VALUE_THRESH:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

# PHYLOGENETIC THREE

The Bio.Phylo aims to provide a common way to work with phylogenetic trees independently of the source data format.

Bio.Phylo is described in an open-access journal article(https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-13-209), which you might also find helpful.


In [ ]:
from Bio import AlignIO

alignment = AlignIO.read("PF08792_seed.sth", "stockholm")

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceCalculator

#calculate the distance matrix
calculator = DistanceCalculator('identity')
#adds distance matrix to the calculator object and returns it
dm = calculator.get_distance(alignment)
print(dm)

and finally, we can construct a phylogenetic tree from the pairwise distances between all the sequences.

In [ ]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor

#initialize a DistanceTreeConstructor object based on our distance calculator object
constructor = DistanceTreeConstructor(calculator)

#build the tree
upgma_tree = constructor.build_tree(alignment)

In [ ]:
print(upgma_tree)

And let's use the Phylo module to visualize the result!

In [ ]:
%matplotlib inline
from Bio import Phylo
import matplotlib.pyplot
#draw the tree
Phylo.draw(upgma_tree)

The function draw support the display of different colors and branch widths in a tree

In [ ]:
tree = upgma_tree.as_phyloxml()

In [ ]:
tree.root.color = (128, 128, 128)

Colors for a clade are treated as cascading down through the entire clade, so when we colorize the root here, it turns the whole tree gray. We can override that by assigning a different color lower down on the tree.



In [ ]:
mrca = tree.common_ancestor({"name": "Inner10"})
mrca.color = "salmon"


Color a clade (branch):

In [ ]:
tree.clade[0, 1].color = "blue"

In [ ]:
Phylo.draw(tree)